# A Short Tour of Julia

In the first part of the lecture we will see:

- Managing data with ```DataFrames.jl```
- Neural networks with ```Flux.jl```
- Benchmarking with ```BenchmarkTools.jl```
- Plotting: ```Plots.jl``` and others libraries

In [1]:
using Pkg
Pkg.activate(".")

  Activating new project at `~/codici/codici_dssc/julia_course`


In [2]:
using DataFrames
using Plots
using Statistics

ArgumentError: ArgumentError: Package DataFrames not found in current path.
- Run `import Pkg; Pkg.add("DataFrames")` to install the DataFrames package.

## Managing Data with DataFrames

DataFrames on Julia are similar to dataframes in R and Pandas dataframes in Python

Let us start by creating an _empty_ dataframe. We will add/load new data

In [ ]:
df = DataFrame()

An empty dataframe is quite boring, let us generate some data:
- $x$ positions from $1$ to $10$
- a first random $y$ coordinate (uniform in $[0,1)$)
- a second random $y$ coordinate ($N(0,1)$)

In [ ]:
data = hcat(collect(1:10), rand(10, 1), randn(10, 1))
df = DataFrame(data, :auto)

We can rename the columns of the dataframe by passing a vector of Strings of Symbols to the ```rename!``` function (notice the ```!```)

In [ ]:
names = [:x, :y₁, :y₂]
rename!(df, names)

We could have also added the names during the creation of the dataframe

In [ ]:
df = DataFrame(data, [:x, :y₁, :y₂])

We can access the different columns of the dataframe by name

In [ ]:
df.y₁ # An array of Float64

In [ ]:
df[:, :x] # This looks more similar to array/dictionary access 

In [ ]:
df."x" # We can even use strings...

We can add a new column by simply assigning a vector of suitable length ($10$ elements in this case)

In [ ]:
df.x₂ = 10*rand(10)
df

We might want to rename and reorder the columns.

We can do this via the ```rename``` and ```select!``` functions:

In [ ]:
rename!(df, :x => :x₁)
select!(df, r"x", :) # group all columns matching the regexp "x" before all the rest (:)

Let us get some statistics on this data via the ```describe``` function

In [ ]:
describe(df)

We can select which statstics to get via additional arguments to the ```describe``` function.

If we need a matrix, instead of a dataframe, simply using the ```Matrix()``` constructor works:

In [ ]:
Matrix(df)

Before moving on, let us start with our first scatter plot.

- ```scatter``` creates a new plot
- ```scatter!``` adds to the existing plot

In [ ]:
scatter(df.x₁, df.y₁, label="data 1")
scatter!(df.x₂, df.y₂, label="data 2")

### A few notes on plotting in Julia

There are multiple packages that can be used for plotting in Julia:

- ```Plots.jl```: the "main" Julia plotting library with multiple backends (including in JavaScript)
- ```PyPlots.jl```: wrapper for Python's matplotlib
- ```Gadfly.jl```: promising package, inspired by ggplot

### Manipulation of Dataframes

In [ ]:
df = DataFrame()

for i ∈ 1:10^5
    elem::Vector{Float64} = []
    while sum(elem) ≤ 1
        push!(elem, rand())
    end
    push!(df, (id = i, length = length(elem), elements = elem))
end

df

Maybe we want to add the sum of all the elements in each list as an additional column in out dataframe.

Notice that ```ByRow``` indicates that the function is applied to each row of the column, not to the entire column

In [ ]:
transform!(df, :elements => ByRow(sum))

```elements_sum``` is not a good name. Let us delete the column and create it again with a different name (without using ```rename!``` which would be better)

In [ ]:
select!(df, :id, :length, :elements)
transform!(df, :elements => ByRow(sum) => :sum)

Let us find the average length of the list of elements

In [ ]:
mean(df.length)

It is possible to prove that the expecte value is the constant $e$:

In [ ]:
MathConstants.e

We could have used ```combine``` to _combine_ all elements of a column in a single value

In [ ]:
combine(df, :length => mean)

Let us explore how we can group the different rows of the dataframe using the ```groupby``` function.

In [ ]:
grouped_df = groupby(df, :length, sort=true) |> x -> combine(x, :length => length => :num_elems)

In [ ]:
histogram(df.length, yaxis = :log, bar_width = 0.75, title = "number of sequences", key=false)

## Building a Neural Network

We are going to build a simple neural network from scratch, then we are going to use the facilities provided by ```Flux.jl``` to help us build and train neural networks.

In [ ]:
using Flux

### Automatic gradient computation

Let us define a function of which we want to compute the derivative:

In [3]:
f(x) = 3x^3 + 2x^2 + 5

f (generic function with 1 method)

We can compute the derivative by using the ```gradient``` function:

In [4]:
derivative_f(x) = gradient(f, x)[1]

derivative_f (generic function with 1 method)

Let us plot both $f$ and its derivative:

In [5]:
x_vals = -5:0.01:5

plot(x_vals, f.(x_vals), label="f(x)")
plot!(x_vals, derivative_f.(x_vals), label="f'(x)")

UndefVarError: UndefVarError: plot not defined

Notice that we expect $9x^2 + 4x$ as a derivative and a good automatic differentiation engine will actually write the code corresponding to it

In [ ]:
@code_llvm derivative_f(3.0) # we expect 9x^2 + 4x

For an introduction to automatic diffentiation the [wikipedia page](https://en.wikipedia.org/wiki/Automatic_differentiation) provides a good overview.

For one of the automatic differentiation framework in Julia that is used in Flux see [Zigote.jl](https://github.com/FluxML/Zygote.jl) and the paper describing how automatic differentiation is performed on [arXiv](https://arxiv.org/abs/1810.07951).

### Neural Networks from scratch

For a general introduction to machine learning a quick read is [The hundred-page machine learning book](http://themlbook.com/wiki/doku.php) where all chapter are available online. For a more in-depth course on neural networks and deep learning, we refer to the [Deep Learning course](https://atcold.github.io/pytorch-Deep-Learning/) by Yann LeCun and Alfredo Canziani.

Let us build a simple fully connected layer (i.e., a simple linear function) with two inputs and one output:

In [ ]:
W = rand(1, 2) .- 0.5;
b = rand(1) .- 0.5;

The output of this linear function is $Wx + b$:

In [ ]:
simple_layer(x) = W*x .+ b

The error between the expected outputs $y = (y_1, \ldots, y_n)$ and the outputs $\hat{y} = (\hat{y}_1, \ldots, \hat{y}_n)$ given by the layer is $\frac{1}{n} \sum_{i=1}^n (y_i - \hat{y}_i)^2$:

In [ ]:
function loss(x, y)
    ŷ = simple_layer(x)
    mean((y .- ŷ).^2)
end

We can compute the gradient by using the ```gradient``` function made available by Flux, and we can decide to derive whith respect to what parameters by using ```Flux.params```

In [ ]:
d_simple(x, y) = gradient(() -> loss(x,y), Flux.params(W, b))

println(d_simple([2, 3], 4)[W])
println(d_simple([2, 3], 4)[b])

How can we train this simple neural network/linear function? by using gradient descent. Notice that here we use ```global``` like in Python to modify a variable in the global scope

In [ ]:
function train!(X, Y; η=0.1)
    grad = d_simple(X,Y)
    W̃ = grad[W]
    b̃ = grad[b]
    global W = W - η*W̃
    global b = b - η*b̃
end

Let us generate a very simple training set as $100$ random point where the target value is actually a linear function of first component plus a gaussian noise:

In [ ]:
simple_X = rand(2, 100)
simple_y = [10*i + randn()*0.05 + 3 for i ∈ simple_X[1,:]];

Let us visualize the data in 3D:

In [ ]:
function show_data()
    scatter3d(simple_X[1,:], simple_X[2,:], simple_y, label="target")
    simple_ŷ = simple_layer(simple_X)
    scatter3d!(simple_X[1,:], simple_X[2,:], reshape(simple_ŷ, (100,)), label="predicted")
end

show_data()

We can now train for a few epochs the network, printing the loss before and after the training:

In [ ]:
println("Loss before training : $(loss(simple_X, simple_y))")
for _ in 1:1000
    train!(simple_X, simple_y)
end
println("Loss after training: $(loss(simple_X, simple_y))")

In [ ]:
show_data()

### MNIST

Let us download the MNIST dataset, which contains $60,000$ images of handwritten digits as $28x28$ greyscale images.

You can find it within the ```MLDatasets``` package

In [ ]:
using MLDatasets
images = MLDatasets.MNIST().features
labels = MLDatasets.MNIST().targets;

Let us see one of the images:

In [ ]:
images[:,:,1]

The first image has label: 

In [ ]:
labels[1]

Some standard preprocessing:

- encoding the labels as one-hot vectors of $10$ elements
- For this example we will use only $1,000$ images instead of $60,000$
- change the type of the images as arrays of ```Float64``` and the shape of the input as $28 \times 28 \times \textit{ num channels } \times \textit{ num samples}$
- prepare the minibatches.

In [ ]:
n_images = 1000

Y = Flux.onehotbatch(labels[1:n_images], 0:9);

X = Float32.(reshape(images[:,:,1:n_images], (28, 28, 1, n_images)))

batches = Flux.Data.DataLoader((X, Y), batchsize=32);

We can now build our model as a convolutional neural network with:

- convolutional layers, with a $3x3$ kernel, a padding of $1$ in all directions and with _input channels_ ```=>``` _output channels_
- max pooling layers
- a dense layer with $288$ inputs and $10$ outpus followed by a softmax layer

In [ ]:
model = Chain(
    Conv((3,3), 1 => 16, relu, pad=(1,1)),
    MaxPool((2,2)),
    Conv((3,3), 16 => 32, relu, pad=(1,1)),
    MaxPool((2,2)),
    Conv((3,3), 32 => 32, relu, pad=(1,1)),
    MaxPool((2,2)),
    Flux.flatten,
    Dense(288, 10, identity),
    softmax)

We can define the loss as the _crossentropy_ loss. Notice that the model is now included in the definition of the loss function.

In [ ]:
function loss(x, y)
    ŷ = model(x)
    Flux.Losses.crossentropy(y, ŷ)
end

In addition to the loss function, we are interested in the accuracy of the prediction:

In [ ]:
function accuracy(x, y)
    mean(Flux.onecold(model(x)) .== Flux.onecold(y))
end

We decide which optimizer to use (e.g., ADAM, ADAGrad, etc.)

In [ ]:
optim = Flux.ADAM()

We also define a callback function to be called at most once every ```n_seconds``` during training to print the current value of the loss

In [ ]:
n_seconds = 5
cb = Flux.throttle(() -> println("Current loss: $(loss(X, Y))"), n_seconds)

How good is our untrained network?

In [ ]:
accuracy(X, Y)

We can now train the network. Notice that we also have a macro ```Flux.@epochs num_epochs code``` available

In [ ]:
for i ∈ 1:10
    println("Epoch $i")
    Flux.train!(loss, Flux.params(model), batches, optim, cb=cb)
end

We can see that our accuracy has improved (but not by a lot, we had a very short learning phase):

In [ ]:
accuracy(X, Y)

## Benchmarking

We have seen a few ways of exploring how much time a certain operation requires in Julia, using the ```@time``` or the ```@timed``` macros.

Let us start by benchmarking this function ```my_sum``` with a $10^6$ vector of random elements

In [ ]:
function my_sum(v)
    s = 0.0 # zero(eltype(v)) # would be better since it will use the "correct" zero
    for x ∈ v
        s += x
    end
    s
end

In [ ]:
rand_vec = rand(10^6);

Let us time the function using the ```@time``` macro:

In [ ]:
@time my_sum(rand_vec)

In [ ]:
@time my_sum(rand_vec)

Since Julia is JIT-compiled the first execution includes the compilation and might not be representative of the successive execution. Furthermore, we need more than one execution to get some significant result!

We can use the ```BenchmarkTools.jl``` package

In [ ]:
using BenchmarkTools

We now have the ```@benchmark``` macro that executes the code multiple times

In [ ]:
@benchmark my_sum($rand_vec)

Notice that there is still some overhead, since we are calling python code from Julia.

But what about the Julia native sum implementation?

In [ ]:
@benchmark sum($rand_vec)

## And now a small break

In [ ]:
scatter(randn(3,2000) .+ [-3,0,3], randn(3, 2000) .+ [-3,2,-3], 
        c=palette(:default)[2:4], key=:none, grid=false, showaxis=false,
        ticks=false, size=(600,600), markerstrokewidth=0)

# A Short Tour of Julia

In this part of the lecture we will explore:

- Differential equations with ```DifferentialEquations.jl```
- Probabilistic programming with ```Turing.jl```
- Distributed computation

## Differential Equations

The ```DifferentialEquations.jl``` package makes easy to define and solve multiple kinds of differential equations, with multiple solvers available depending on the kind of equation.

In [ ]:
using DifferentialEquations
using Plots

Let us define a simple ordinary differential equation 
$$ \frac{du}{dt} = \alpha u $$
where $\alpha$ is a parameter

In [ ]:
α = 1.02
f(u, p, t) = α*u

By default in the definition of the ODE there are:
- The variable $u$.
- A collection of parameters to the ODE (we will show how to use them shortly).
- The time $t$.

We now need to define the initial state $u_0$ and the time interval for which we want to solve our ODE

In [ ]:
u0 = 0.03
tspan = (0.0, 1.0);

We can now define an ODE problem, which is entirely determined by the ODE, the initial conditions, and the timespan:

In [ ]:
prob = ODEProblem(f, u0, tspan)

To obtain a solution we can simply call ```solve```. As additional arguments we can specify:

- the solver to use (see the [documentation](https://diffeq.sciml.ai/v6.16/solvers/ode_solve/#ode_solve)).
- relative and absolute tolerances (keyword arguments ```reltol``` and ```abstol```, respectively).
- we might also only give a _hint_ to select the solver with, for example, ```alg_hints=[:stiff]```.

In [ ]:
sol = solve(prob)

We can now plot the solution (and the analytical solution for comparison):

In [ ]:
plot(sol, lw=3, xaxis="Time (t)", yaxis="u(t)", legend=false)
plot!(0:0.01:1, t->u0*exp(α*t), lw=3, ls=:dash)

Notice that the structure ```sol``` can be used as a function that, for values that were not computed, provides an interpolation:

In [ ]:
[sol(i) for i ∈ 0:0.1:1]

The solution structure also contains information about the pair $(t,u)$ that were computed during the solution process in ```sol.t``` and ```sol.u```:

In [ ]:
println("t = $(sol.t)\nu = $(sol.u)")

### Parameters

We had one argument of $f$ that was not used: ```p```.

```p``` can be an strucure of any type containing the parameters that we want to use in the differential equation.

In [ ]:
g(u, p, t) = p * u

The value of the parameters can be passed as an argument of the ODE problem

In [ ]:
prob = ODEProblem(g, u0, tspan, 1.03)

Everyting continues to work as before

In [ ]:
sol = solve(prob, Tsit5(), reltol=1e-8, abstol=1e-8)
plot(sol,lw=3, xaxis="Time (t)", yaxis="u(t)", legend=false)
plot!(sol.t, t->u0*exp(1.03*t), lw=3, ls=:dash)

### In-place

Until now we had ```In-place: false``` for all our ODE problem. This means that there is a new allocation every time we need to compute a new value of ```f```. We can avoid the allocation by using an ```in-place``` definition where the value is returned by modifing one of the arguments

In [ ]:
function f!(du, u, p, t)
    du[1] = p * u[1]
end

In [ ]:
prob_inplace = ODEProblem(f!, [u0], tspan, α)

There are two things to notice:

- One additional argument (the first argument) to $f!$, which is the value to be modified.
- The value of ```In-place``` when the ```ODEProblem``` is defined is now ```true```

In [ ]:
sol_inplace = solve(prob_inplace)
plot(sol_inplace, lw=3, xaxis="Time (t)", yaxis="u(t)", legend=false)
plot!(0:0.01:1, t->u0*exp(α*t), lw=3, ls=:dash)

### Lotka-Volterra Equations

The Lotka-Volterra equations are a model of population dynamics where there are two species, one acting as prey and one as predator:

$$
\frac{dx}{dt} = \alpha x - \beta xy \\
\frac{dy}{dt} = \delta xy - \gamma y
$$

Here $x$ is the size of the prey population and $y$ the size of the predator population and:

- $\alpha$ is the rate at which the preys increase in number.
- $\beta$ is the rate at which the preys are killed by the predators.
- $\gamma$ is the rate at which the predators die or leave the territory.
- $\delta$ is the rate at which the population of predators increases, that depends on how many preys they are able to catch.

We can now define an (in-place) function for the Lotka-Volterra equations:

In [ ]:
function lotka_volterra!(du, u, p, t)
    α, β, γ, δ = p
    du[1] = α*u[1] - β*u[1]*u[2]
    du[2] = δ*u[1]*u[2] - γ*u[2]
end

We now model a populations starting at the same size, with a default set of parameters (recall that we can use named tuples to improve readability):

In [ ]:
u0 = [10, 10]
p = (α = 1.1, β = 0.4, γ = 0.4, δ = 0.1)
tspan = (0.0, 100.0)
prob_lv = ODEProblem(lotka_volterra!, u0, tspan, p)

Finding the evolution in time of the two populations can still be obtained by simply calling the ```solve``` function

In [ ]:
sol_lv = solve(prob_lv);

In [ ]:
plot(sol_lv.t, [u[1] for u ∈ sol_lv.u], label="prey")
plot!(sol_lv.t, [u[2] for u ∈ sol_lv.u], label="predator")
xlabel!("time")
ylabel!("population size")

### Others Types of Differential Equations

The library ```DifferentialEquations.jl``` can also manage many kinds of differential equations (from the [documentation](https://diffeq.sciml.ai/v6.16/)):

- Discrete equations (function maps, discrete stochastic (Gillespie/Markov) simulations)
- Ordinary differential equations (ODEs)
- Split and Partitioned ODEs (Symplectic integrators, IMEX Methods)
- Stochastic ordinary differential equations (SODEs or SDEs)
- Random differential equations (RODEs or RDEs)
- Differential algebraic equations (DAEs)
- Delay differential equations (DDEs)
- Stochastic delay differential equations (SDDEs)
- Mixed discrete and continuous equations (Hybrid Equations, Jump Diffusions)
- (Stochastic) partial differential equations ((S)PDEs) (with both finite difference and finite element methods)

We will see as example the SIR model, where the jumps are discrete.

### The SIR model

The SIR model is used to simulate the effect of the diffusion of a disease inside a population. The states of each individual can be:

- Susceptible
- Infected
- Recovered

A susceptible person in contact with an infected one can become infected (with a certain rate $c_1$). An infected person transition to a recovered state with a certain rate $c_2$.

We import the ```Catalyst.jl``` module that allow us to write in a compact way a reaction network. I.e., we want to write that $s + i \rightarrow 2 i$ with a certain rate $c_1$ and $i \rightarrow r$ with a certain rate $c_2$

In [ ]:
using Catalyst

```Catalyst``` makes writing these reactions quite simple:

In [ ]:
sir_model = @reaction_network begin
    c1, s + i --> 2i
    c2, i --> r
    end c1 c2;

We can now set the initial conditions:

In [ ]:
p = (1e-4, 0.01)
tspan = (0.0, 300.0)
u0 = [999, 1, 0] # 999 susceptible, 1 infected, 0 recovered
prob_sir = DiscreteProblem(sir_model, u0, tspan, p)

And find the dynamics of the disease:

In [ ]:
prob_jump = JumpProblem(sir_model, prob_sir, Direct())
sol_sir = solve(prob_jump, SSAStepper());

In [ ]:
plot(sol_sir)

### Packages to check

The following packages are related to ```DifferentialEquations.jl``` and can be interesting for specific applications:

- ```DiffEqFlux.jl```, to create Neural ODE.
- ```diffeqpy``` and ```diffeqr``` makes the solvers of ```DifferentialEquations.jl``` available to Python and R, respectively.

## Probabilistic Programming

One of the main libraries in Julia for probabilistic programming is ```Turing.jl```.

Programming works with a known model with known parameters to generate some data.

Probabilistic programming is the case when the data and the model are known but we ignore the parameters.

#### Some references

[An Introduction to Probabilistic Programming](https://arxiv.org/abs/1809.10756)

Let us import ```Turing.jl``` plus a library of distributions, and a library to work with Makow chain Monte Carlo

In [ ]:
using Turing
using Distributions
using MCMCChains

A model (prefixed with the macro ```@model```) is a function. The function can then be used to condition the model on the data.

Here we model a coin flip (example taken from this [tutorial](https://turing.ml/dev/tutorials/0-introduction/)) where we do not know the probability $p$ of landing on head or tail.

Here, we have $y_1, \ldots, y_n$ samples that we know that theu will be distributed according to a Bernoulli distribution with parameter $p$.

In [ ]:
@model function coin(y)
    p ~ Beta(1,1)
   
    for i ∈ 1:length(y)
        y[i] ~ Bernoulli(p)
    end
end;

We can now generate $100$ samples from a fair coin:

In [ ]:
data = rand(Bernoulli(0.5), 100);

We can then use the model to estimate the probability $p$ of landing on tail given that data that we have:

In [ ]:
ϵ = 0.05
τ = 10
iterations = 1000
chain = sample(coin(data), HMC(ϵ, τ), iterations)

In [ ]:
histogram(chain[:p], label=:none)

## Distributed Computation

We are going to explore how it is possible to easily distribute work across multiple processes (and, possibly, machines) in Julia.

To see size of computations possible in Julia see the presentation [Celeste.jl: Petascale Computing in Julia](https://www.youtube.com/watch?v=uecdcADM3hY).

First of all, we import the ```Distributed``` package

In [ ]:
using Distributed
using BenchmarkTools

We can use the function ```nprocs``` to see the number of active Julia processes

In [ ]:
nprocs()

Additional worker processes can be added with the ```addproc``` function, which return an array of integer ids representing the newly created processes

In [ ]:
addprocs(1)

As we can see, the number of Julia processes is now increased:

In [ ]:
nprocs()

We will see a collection of macros, structures, and functions that are used for computation among multiple processes:

- ```@everywhere``` to execute a block of code in all processes.
- ```@spawnat``` to execute a function in a specific process.
- ```Future``` and ```fetch``` the result 
- ```@sync```
- ```@distributed```

Our test case will be the following function to approximate $\pi$ via a uniform sampling in $[0,1]^2$.

In [ ]:
function approx_pi(n)
    s = 0
    for i ∈ 1:n
        p = (rand(), rand())
        s += p[1]^2 + p[2]^2 <= 1
    end
    4s/n
end

In [ ]:
approx_pi(1_000_000)

Let us benchmark it using ```BenchmarkTools.jl```

In [ ]:
@btime approx_pi(10^7)

We can now distribute the computation:

- Define a function ```approx_pi_sum``` on every process
- Split the $n$ points to sample in equal-length chunks and distribute it across processes with ```@spawnat```
- Each computation returns a ```Future```
- Wait for each computation to finish with ```@sync```
- Fetch the results of the computation with ```fetch```
- Finally compute this approximation of $\pi$

In [ ]:
@everywhere function approx_pi_sum(n)
    s = 0
    for i ∈ 1:n
        p = (rand(), rand())
        s += p[1]^2 + p[2]^2 <= 1
    end
    s
end

function compute_pi(n)
    np = nprocs()
    partial_sums = Vector(undef, np)
    k = n ÷ np
    @sync for i ∈ 1:np
        partial_sums[i] = @spawnat i approx_pi_sum(k)
    end
    missings = n - np*k
    s = approx_pi_sum(missings)
    4*(sum(fetch.(partial_sums)) + s)/n
end
     

In [ ]:
@btime compute_pi(10^7)

Since this is so common, we can use the ```@distributed``` macros, that distribute the computation of a ```for``` cycle across all processes and reduce it with a given function

In [ ]:
function distributed_pi(n)
    sum = @distributed (+) for i ∈ 1:n
            p = (rand(), rand())
            p[1]^2 + p[2]^2 <= 1
    end
    4*sum/n
end

In [ ]:
distributed_pi(10^7)

In [ ]:
@btime distributed_pi(10^7)

### See Also

- ```MPI.jl``` - A Julia interface to MPI.
- ```DistributedArrays.jl``` - Arrays that are stored on multiple processes possibly on different machines.
- JuliaGPU and ```CUDA.jl``` to perform computations on GPU. Notice that ```Flux``` can use GPUs and also many other packages can work with them.